In [0]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np

import os 
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
from skimage import color, exposure

%load_ext tensorboard

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix3/'

/content/drive/My Drive/Colab Notebooks/matrix/matrix3


In [0]:
!pwd

/content/drive/My Drive/Colab Notebooks/matrix/matrix3


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim  == 1:y_train = to_categorical(y_train)
if y_test.ndim  == 1:y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
print(input_shape)
num_classes = y_train.shape[1]
print(num_classes)

(32, 32, 3)
43


In [0]:
def train_model(model, X_train, y_train, X_test, y_test, params_fit ={} ):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
	
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(X_train, 
            y_train,
            batch_size = params_fit.get('batch_size', 128),
            epochs = params_fit.get('epochs', 5),
            verbose = params_fit.get('verbose', 1),
            validation_data = params_fit.get('validation_data', (X_test, y_test)),
            callbacks = [tensorboard_callback]
          )
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score ):
	
  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def train_and_predict(model, X_train, y_train,  X_test, y_test):
  model_trained = train_model (model, X_train, y_train, X_test, y_test)
  return predict(model_trained, X_test, y_test)

In [0]:
def get_cnn_v5 (input_shape, num_classes):
	return Sequential([
                    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape = input_shape),
                    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
                    MaxPool2D(),
                    Dropout(0.3),

                    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),
                    
                    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),
                    
                    Flatten(),
                    Dense ( 1024, activation = 'relu'),
                    Dropout(0.3),

                    Dense ( 1024, activation = 'relu'),
                    Dropout(0.3),

                    Dense ( num_classes, activation = 'softmax'),
                    ])
  
get_cnn_v5(input_shape, num_classes).summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 6, 64)         

In [0]:
train_and_predict(get_cnn_v5(input_shape, num_classes), X_train, y_train,  X_test, y_test)

Epoch 1/5
272/272 [==============================] - 10s 38ms/step - loss: 2.6914 - accuracy: 0.2518 - val_loss: 1.4328 - val_accuracy: 0.5408
Epoch 2/5
272/272 [==============================] - 10s 35ms/step - loss: 0.9237 - accuracy: 0.7040 - val_loss: 0.3591 - val_accuracy: 0.8971
Epoch 3/5
272/272 [==============================] - 10s 35ms/step - loss: 0.3908 - accuracy: 0.8796 - val_loss: 0.1308 - val_accuracy: 0.9610
Epoch 4/5
272/272 [==============================] - 10s 35ms/step - loss: 0.2372 - accuracy: 0.9291 - val_loss: 0.1450 - val_accuracy: 0.9580
Epoch 5/5
272/272 [==============================] - 9s 35ms/step - loss: 0.1863 - accuracy: 0.9462 - val_loss: 0.1487 - val_accuracy: 0.9685


0.9684807256235828

In [0]:
def get_model(params):#, input_shape = input_shape, num_classes = num_classes):
	return Sequential([
                    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape = input_shape),
                    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_one']),
                    
                    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_two']),

                    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
                    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_three']),
                    
                    Flatten(),
                    Dense ( 1024, activation = 'relu'),
                    Dropout(params['dropout_dense_block_one']),

                    Dense ( 1024, activation = 'relu'),
                    Dropout(params['dropout_dense_block_two']),

                    Dense (num_classes, activation = 'softmax'),
                    ])

#get_model(space).summary() 
#get_model(input_shape, num_classes).summary()

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
    X_train, 
    y_train,
    batch_size = int (params.get('batch_size', 128)),
    epochs = 5,
    verbose = 0
  )
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [0]:
space = { 
    'batch_size': hp.quniform('batch_size', 50, 200, 20),
    'dropout_cnn_block_one': hp.quniform('dropout_cnn_block_one', 0, 0.6, 0.1),
    'dropout_cnn_block_two': hp.quniform('dropout_cnn_block_two', 0, 0.6, 0.1),
    'dropout_cnn_block_three': hp.quniform('dropout_cnn_block_three', 0, 0.6, 0.1),
    'dropout_dense_block_one': hp.quniform('dropout_dense_block_one', 0, 0.6, 0.1),
    'dropout_dense_block_two': hp.quniform('dropout_dense_block_two', 0, 0.6, 0.1)
}

In [0]:
best =fmin(
    func_obj, 
    space, 
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 60.0, 'dropout_cnn_block_one': 0.2, 'dropout_cnn_block_three': 0.4, 'dropout_cnn_block_two': 0.0, 'dropout_dense_block_one': 0.6000000000000001, 'dropout_dense_block_two': 0.1}
accuracy=0.9689342379570007
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.5, 'dropout_cnn_block_three': 0.30000000000000004, 'dropout_cnn_block_two': 0.30000000000000004, 'dropout_dense_block_one': 0.30000000000000004, 'dropout_dense_block_two': 0.2}
accuracy=0.9195011258125305
{'batch_size': 60.0, 'dropout_cnn_block_one': 0.4, 'dropout_cnn_block_three': 0.2, 'dropout_cnn_block_two': 0.4, 'dropout_dense_block_one': 0.5, 'dropout_dense_block_two': 0.30000000000000004}
accuracy=0.9537414908409119
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.2, 'dropout_cnn_block_three': 0.0, 'dropout_cnn_block_two': 0.1, 'dropout_dense_block_one': 0.0, 'dropout_dense_block_two': 0.6000000000000001}
accuracy=0.9628117680549622
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.4, 'dropout_cnn_block_three': 0.300

In [0]:
#BEST PARAMS
print(best)
#func_obj(best_params)

In [0]:
best = {'batch_size': 160.0,
 'dropout_cnn_block_one': 0.3,
 'dropout_cnn_block_two': 0.1,
 'dropout_cnn_block_three': 0.4,
 'dropout_dense_block_one': 0.2,
 'dropout_dense_block_two': 0.5
 }